# Day 3: Disney Parks Guest Spending Behavior

You are a data analyst working with the Disney Parks revenue team to understand nuanced guest spending patterns across different park experiences. The team wants to develop a comprehensive view of visitor purchasing behaviors. Your goal is to uncover meaningful insights that can drive personalized marketing strategies.

In [ ]:
import pandas as pd
import numpy as np

fct_guest_spending_data = [
  {
    "guest_id": 1,
    "visit_date": "2024-07-05",
    "amount_spent": 50,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-07-06",
    "amount_spent": 30,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-07-10",
    "amount_spent": 20.5,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 4,
    "visit_date": "2024-07-12",
    "amount_spent": 40,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-07-15",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 5,
    "visit_date": "2024-07-20",
    "amount_spent": 60,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 6,
    "visit_date": "2024-07-25",
    "amount_spent": 25,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-03",
    "amount_spent": 55,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-08-15",
    "amount_spent": 45,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-05",
    "amount_spent": 22,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 2,
    "visit_date": "2024-08-20",
    "amount_spent": 38,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 7,
    "visit_date": "2024-08-10",
    "amount_spent": 15,
    "park_experience_type": "Character Meet"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-25",
    "amount_spent": 28,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 3,
    "visit_date": "2024-08-27",
    "amount_spent": 32,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-02",
    "amount_spent": 65,
    "park_experience_type": "Attraction"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-05",
    "amount_spent": 50,
    "park_experience_type": "Retail"
  },
  {
    "guest_id": 9,
    "visit_date": "2024-09-15",
    "amount_spent": 40,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 10,
    "visit_date": "2024-09-20",
    "amount_spent": 70,
    "park_experience_type": "Entertainment"
  },
  {
    "guest_id": 1,
    "visit_date": "2024-09-25",
    "amount_spent": 35,
    "park_experience_type": "Dining"
  },
  {
    "guest_id": 8,
    "visit_date": "2024-09-28",
    "amount_spent": 10,
    "park_experience_type": "Character Meet"
  }
]
fct_guest_spending = pd.DataFrame(fct_guest_spending_data)


## Question 1

What is the average spending per guest per visit for each park experience type during July 2024? Ensure that park experience types with no recorded transactions are shown with an average spending of 0.0. This analysis helps establish baseline spending differences essential for later segmentation.

In [ ]:
# Convert to datetime and extract year-month
fct_guest_spending["visit_date"] = pd.to_datetime(fct_guest_spending["visit_date"], errors="coerce")
fct_guest_spending["year_month"] = fct_guest_spending["visit_date"].dt.to_period("M")

# Filter for July 2024
july_data = fct_guest_spending[fct_guest_spending["year_month"] == "2024-07"]

# All experience types
all_experiences = fct_guest_spending["park_experience_type"].unique()

# Average spending per visit
july_grouped = (
    july_data.groupby("park_experience_type")["amount_spent"]
    .mean()
    .reset_index(name="avg_spending_per_visit")
)

# Ensure all experience types are included, fill missing with 0.0
july_grouped = (
    july_grouped.set_index("park_experience_type")
    .reindex(all_experiences, fill_value=0.0)
    .reset_index()
)

print(july_grouped)

## Question 2

For guests who visited our parks more than once in August 2024, what is the difference in spending between their first and their last visit? This investigation, using sequential analysis, will reveal any shifts in guest spending behavior over multiple visits.

In [ ]:
# Filter for August 2024
aug_data = fct_guest_spending[
    (fct_guest_spending["visit_date"].dt.year == 2024) &
    (fct_guest_spending["visit_date"].dt.month == 8)
]

# Ensure sorted
aug_data = aug_data.sort_values(["guest_id", "visit_date"])

# Get first and last visit spending per guest
first_last_spending = (
    aug_data.groupby("guest_id")
    .agg(
        first_spent=("amount_spent", "first"),
        last_spent=("amount_spent", "last"),
        visit_count=("visit_date", "count")
    )
    .reset_index()
)

# Keep only guests with multiple visits
first_last_multi = first_last_spending[first_last_spending["visit_count"] > 1]

# Add column for difference (last - first)
first_last_multi["spending_diff"] = (
    first_last_multi["last_spent"] - first_last_multi["first_spent"]
)

print(first_last_multi.head())

## Question 3

In September 2024, how can guests be categorized into distinct spending segments such as Low, Medium, and High based on their total spending? Use the following thresholds for categorization: 
-Low: Includes values from $0 up to, but not including, $50.
-Medium: Includes values from $50 up to, but not including, $100.
-High: Includes values from $100 and above. 
Exclude guests who did not make any purchases in the period.

In [ ]:
# Filter for August 2024
sept_data = fct_guest_spending[
    (fct_guest_spending["visit_date"].dt.year == 2024) &
    (fct_guest_spending["visit_date"].dt.month == 9)
]

# Keep only guests with >0 spending
active_guests = sept_data[sept_data["amount_spent"] > 0]

# Step 1: Aggregate total spending per guest
total_spending = (
    active_guests.groupby("guest_id")["amount_spent"]
    .sum()
    .reset_index(name="total_spent")
)

# Step 2: Define bins and labels
bins = [0, 50, 100, float("inf")]  # upper bound is exclusive
labels = ["Low", "Medium", "High"]

# Assign labels
total_spending["spending_category"] = pd.cut(
    total_spending["total_spent"],
    bins=bins,
    labels=labels,
    right=False  # upper bound is not included
)

print(total_spending.head())

Made with ❤️ by [Interview Master](https://www.interviewmaster.ai)